# 0. Configuration

In [2]:
RANDOM_STATE = 42
TEST_SIZE = 0.3

# 1. Modules and Functions

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# 2. Main

## 2.1. Data preperation

Making an artificial dataset for Anomaly Detection (the reason why it is imbalanced)

In [4]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                          weights=[0.9, 0.1], flip_y=0, random_state=RANDOM_STATE)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size = TEST_SIZE, 
    stratify = y, 
    random_state = RANDOM_STATE
)

print(
    f'Train size is: {X_train.shape} \nTest size is: {X_test.shape}'
)

Train size is: (700, 10) 
Test size is: (300, 10)


## You can ignore it, the very first try

params = {
    "solver" : "lbfgs",
    "max_iter" : 1000,
    "multi_class" : "auto",
    "random_state" : 8888
}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

import mlflow

mlflow.set_experiment("First Experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'report_class_0': report_dict['0']['recall'],
        'report_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression")

# 2.2 Model training  &  2.3 Model evaluation

## Experiment 1: Train Logistic Regression Classifier (liblinear)

In [6]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



## Experiment 2: Train Random Forest Classifier

In [7]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



## Experiment 3: Train XGBoost (logloss)

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



## Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [13]:
pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1


  Obtaining dependency information for imbalanced-learn==0.10.1 from https://files.pythonhosted.org/packages/11/80/911e581a4fc973179e3a48c1272435aa09cce21c12af122c3886d3d35cb5/imbalanced_learn-0.10.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/226.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/226.0 kB ? eta -:--:--
   --- ----------------------------------- 20.5/226.0 kB 330.3 kB/s eta 0:00:01
   ------- ------------------------------- 41.0/226.0 kB 326.8 kB/s eta 0:00:01
   -------------- ------------------------ 81.9/226.0 kB 459.5 kB/s eta 0:00:01
   ------------------------- ------------ 153.6/226.0 kB 706.2 kB/s eta 0:00:01
   -------------------------------------  225.3/226.0 kB 860.2 kB/s eta 0:00:01
   -------------------------------------- 226.0/226.0 kB 811.1 kB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Succe

In [14]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [15]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       270
           1       0.86      0.83      0.85        30

    accuracy                           0.97       300
   macro avg       0.92      0.91      0.92       300
weighted avg       0.97      0.97      0.97       300



# 2.4 Track experiments using MLFlow

In [16]:
models = [
    (    
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_lavel_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [17]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

[16:38:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "use_lavel_encoder" } are not used.



In [18]:
import mlflow

In [20]:
mlflow.set_experiment("Anomaly Detection 1")
mlflow.set_tracking_uri(uri='http://localhost:5000/')

for i, element in enumerate(models):
    model_name = element[0]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2024/12/27 16:41:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/621642682576530848/runs/51fc949541cb409183c54896cbc5bd03
🧪 View experiment at: http://localhost:5000/#/experiments/621642682576530848


2024/12/27 16:41:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/621642682576530848/runs/e0c706632f314dc8971407ec3c74b10f
🧪 View experiment at: http://localhost:5000/#/experiments/621642682576530848


2024/12/27 16:41:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/621642682576530848/runs/e5357f0ade6f4c6ab480481a7c7bfac2
🧪 View experiment at: http://localhost:5000/#/experiments/621642682576530848


2024/12/27 16:41:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/621642682576530848/runs/fa23a60938c14f57bb1905b0aef4a191
🧪 View experiment at: http://localhost:5000/#/experiments/621642682576530848


# Register the Model 

In [ ]:
model_name = "XGB-Smote"
run_id = input("Enter run ID: ")
model_uri = f"runs:/{run_id}/model"

results = mlflow.register_model(
    model_uri, model_name
)

# Load the Model

In [24]:
model_name = "XGB-Smote"
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [27]:
dev_model_uri = f"models:/{model_name}/{model_version}"
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1735325278770, current_stage='None', description='', last_updated_timestamp=1735325278770, name='anomaly-detection-prod', run_id='e08fbfeffc9149a588acff5daa18d01f', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [28]:
model_uri = f"models:/{prod_model}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])